In [1]:
%matplotlib notebook
from pathlib import Path
import pickle
from random import shuffle
import numpy as np
from collections import defaultdict

In [2]:
with open("words.txt",'r') as read_file:
    words = read_file.readlines()
    
for i in range(len(words)):
    words[i] = words[i].strip('\n')

with open('words.pkl', 'wb') as handle:
    pickle.dump(words, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
with open('words.pkl', 'rb') as handle:
    words = pickle.load(handle)

In [3]:
# 2 blank tiles (scoring 0 points)
# 1 point: E ×12, A ×9, I ×9, O ×8, N ×6, R ×6, T ×6, L ×4, S ×4, U ×4
# 2 points: D ×4, G ×3
# 3 points: B ×2, C ×2, M ×2, P ×2
# 4 points: F ×2, H ×2, V ×2, W ×2, Y ×2
# 5 points: K ×1
# 8 points: J ×1, X ×1
# 10 points: Q ×1, Z ×1
scores = dict()
for i in ['A','E','I','O','N','R','T','L','S','U']:
    scores[i] = 1
for i in ['D','G']:
    scores[i] = 2
for i in ['B','C','M','P']:
    scores[i] = 3
for i in ['F','H','V','W','Y']:
    scores[i] = 4
for i in ['K']:
    scores[i] = 5
for i in ['J','X']:
    scores[i] = 8
for i in ['Q','Z']:
    scores[i] = 10
# for i in ['blank']:
#     scores[i] = 0
distributions = dict()
for i in ['N','R','T']:
    distributions[i] = 6
for i in ['L','S','U','D']:
    distributions[i] = 4
for i in ['G']:
    distributions[i] = 3
#removed blanks, would be 2
for i in ['B','C','M','P','F','H','V','W','Y']:
    distributions[i] = 2
for i in ['Q','Z','J','X','K']:
    distributions[i] = 1
for i in ['E']:
    distributions[i] = 12
#should be 9, added one each from blanks
for i in ['A','I']:
    distributions[i] = 10
for i in ['O']:
    distributions[i] = 8

In [4]:
board_array = np.zeros((15,15))
for i in (0,7,14):
    for j in (0,7,14):
        board_array[i,j] = 3000
for i in range(1,5):
    board_array[i,i] = 2000
for i in range(10,14):
    board_array[i,i] = 2000
for i in (5,9):
    board_array[i,i] = 300
for i in (3,11):
    for j in (0,14):
        board_array[i,j] = 200
        board_array[j,i] = 200
for i in range(1,5):
    board_array[i,14-i] = 2000
for i in range(1,5):
    board_array[14-i,i] = 2000
for i in range(10,14):
    board_array[i,i] = 2000
#should be 300 for triple letter
for i in (1,5,9,13):
    for j in (5,9):
        board_array[i,j] = 300
        board_array[j,i] = 300
board_array[7,7] = 0
for i in (6,8):
    for j in (6,8):
        board_array[i,j] = 200
for i in (2,6,8,12):
    for j in (6,8):
        board_array[i,j] = 200
        board_array[j,i] = 200
for j in (3,11):
        board_array[7,j] = 200
        board_array[j,7] = 200
board_array.astype(np.int)
np.save('board', board_array)

In [5]:
class Tile:
    def __init__(self, letter, score, id):
        self.score = score
        self.letter = letter
        self.id = id
    def __repr__(self):
        return '(' + str(self.letter) + ' : ' + str(self.score) + ')'

In [6]:
class Player:
    def __init__(self, name, position):
        self.name = name
        self.score = 0
        self.position = position
        self.rack = list()
        
    def __repr__(self):
        return '(' + str(self.position) + ', ' + str(self.name) + ', score: ' + str(self.score) + ')' 
    
    def draw(self, tiles, board):
        for tile in tiles:
            self.rack.remove(tile)
        board.pool.extend(tiles)
        shuffle(board.pool)
        self.refill(board)
    
    def refill(self, board):
        while len(self.rack)<7:
            if board.pool:
                self.rack.append(board.pool[0])
                board.pool.pop(0)
                
    def show_rack(self):
        print(self.rack)

In [7]:
class Board:
    def __init__(self, num_players):
        self.num_players = num_players
        self.pool = list()
        self.id_dict = dict()
        id = 1
        for key in distributions:
            for i in range(distributions[key]):
                self.pool.append(Tile(key, scores[key],id))
                shuffle(self.pool)
                self.id_dict[id] = Tile(key, scores[key],id)
                id+=1
        for id in board_values:
            self.id_dict[id]= Tile('open',-1,id)
        self.players = dict()
        for i in range(1, num_players+1):
            name = input('What is Player '+str(i)+"'s name?\n")
            self.players[name] = Player(name,i)
        self.board = np.load('board.npy')
        self.board_positions = list()
        self.board_positions.append((7,7))
        
    #def show_board(self):

In [8]:
def show_board(show_board):
    fig, ax = plt.subplots()
    for i in range(15):
        for j in range(15):
            plt.text(j*10,i*10,show_board.id_dict[show.board[i,j]].letter)

In [9]:
from collections import defaultdict
word_lengths = defaultdict(set)
true_lengths = defaultdict(set)
reverse_word_lengths = defaultdict(set)
reverse_true_lengths = defaultdict(set)
for word in words:
    for length in range(1,22):
        if len(word)>=length:
            word_lengths[length].add(word[:length])
        if len(word)==length:
            true_lengths[length].add(word)
        if len(word)>=length:
            reverse_word_lengths[length].add(word[::-1][:length])
        if len(word)==length:
            reverse_true_lengths[length].add(word[::-1])

In [10]:
directions_dict = dict()
def step_left(i,j):
    return (i,j-1)
def step_down(i,j):
    return (i+1,j)
def step_right(i,j):
    return (i,j+1)
def step_up(i,j):
    return (i-1,j)
    
def readcolumn(i,j):
    return (-1,j,2)
def readrow(i,j):
    return (i,-1,1)
directions_dict[0] = step_up
directions_dict[1] = step_right
directions_dict[2] = step_down
directions_dict[3] = step_left
reverse_dict = dict()
reverse_dict[0] = 2
reverse_dict[1] = 3
reverse_dict[2] = 0
reverse_dict[3] = 1
perpen_dict = dict()
perpen_dict[0] = readrow
perpen_dict[1] = readcolumn
perpen_dict[2] = readrow
perpen_dict[3] = readcolumn

In [11]:
board_values = [0,200,300,2000,3000]
def find_openings(input_board):
    positions = input_board.board_positions
    openings = list()
    for i,j in positions:
        if input_board.board[i-1,j] in board_values:
                    openings.append((i,j,0))
        if input_board.board[i,j+1] in board_values:
                    openings.append((i,j,1))
        if input_board.board[i+1,j] in board_values:
                    openings.append((i,j,2))
        if input_board.board[i,j-1] in board_values:
                    openings.append((i,j,3))
    return openings

In [12]:
def find_moves(find_board, openings, tiles):
    moves = list()
    for opening in openings:
        i,j,d = opening
        boardword = readback(find_board,opening)
        boardword = readforward(find_board,opening)
        if d==1 or d==2:
            check = word_lengths
            true_check = true_lengths
        if d==0 or d==3:
            check = reverse_word_lengths
            true_check = reverse_true_lengths
        stops = defaultdict(list)
#         directions_dict[d](i,j)
        add = list(tiles[0])
        word_attempt = read_letters(boardword, forword, add)
        while len(stops[[]])!=7:
            if word_attempt not in check[len(word_attempt)]:
                stem = add[:len(add)-1]
                stop = add[-1]
                stops[stem].append(stop)
                while not find_different(tiles, stem, stops):
                    stop = stem[-1]
                    stem = stem[:len(stem)-1]
                    stops[stem].append(stop)
                add = find_different(tiles, stem, stops)
                word_attempt = read_letters(boardword, forword, add)
            else:
                i,j,d = opening
                for step in range(len(add)):
                    i,j = direction_dict[d](i,j)
                    if i not in range(0,15) or j not in range(0,15):
                        i,j = direction_dict[reverse_dict[d]](i,j)
                        break
                    full = readforward(find_board,perpen_dict[d](i,j))
                    words_in_path = ''.join(full).split()
                    block = 0
                    for word in words_in_path:
                        if len(word)>1 and word not in true_lengths[len(word)]:
                            block = 1
                    if block:
                        stem = add[:len(add)-1]
                        stop = add[-1]
                        stops[stem].append(stop)
                        while not find_different(tiles, stem, stops):
                            stop = stem[-1]
                            stem = stem[:len(stem)-1]
                            stops[stem].append(stop)
                        add = find_different(tiles, stem, stops)
                        word_attempt = read_letters(boardword, forword, add)
                else:
                    if word_attempt in true_check[len(word_attempt)]:
                        moves.append((opening, add))
                    while not add_letter(tiles, stem, stops):
                        stop = stem[-1]
                        stem = stem[:len(stem)-1]
                        stops[stem].append(stop)
                    add = add_letter(tiles, stem, stops)
                    word_attempt = read_letters(boardword, forword, add)
    return moves

In [13]:
def add_letter(tiles, stem, stops):
    poss = tiles.copy()
    for i in stem:
        poss.remove(i)
    for i in stops[stem]:
        poss.remove(i)
    if poss:
        stem.append(poss[0])
        return stem
    else:
        return False
def find_different(tiles, stem, stops):
    poss = tiles.copy()
    for i in stem:
        poss.remove(i)
    for i in stops[stem]:
        poss.remove(i)
    if poss:
        stem.append(poss[0])
        return stem
    else:
        return False

In [14]:
def find_letters(player):
    tile_letters = list()
    for tile in player.rack:
        tile_letters.append(tile.letter)
    return tile_letters
def readback(read_board, opening):
    i,j,d = opening
    m,n = i,j
    readword = list()
    p = directions_dict[reverse_dict[d]]
    while read_board.board[i,j] not in board_values:
        readword.append(read_board.id_dict[read_board[i,j]].letter)
        i,j = p(i,j)
        if i not in range(0,15) or j not in range(0,15):
            break
    boardword  = "".join(readword[::-1])
    return boardword
def readforward(read_board, opening):
    i,j,d = opening
    readword = list()
    i,j = directions_dict[d](i,j)
    while i in range(0,15) and j in range(0,15):
        readword.append(read_board.id_dict[read_board[i,j]].letter)
        i,j = directions_dict[d](i,j)
    return forword

In [15]:
def read_letters(boardword,forword, tiles):
    for i in range(len(tiles)):
        tiles[i] = tiles[i].letter
    i = 0
    attempt = list()
    while tiles:
        if forword[i]!=' ':
            attempt.append(forword[i])
        else:
            attempt.append(tiles[0])
            tiles.pop(0)
        i+=1
    word_attempt = boardword + ''.join(attempt)
    return word_attempt

In [16]:
def check_border(add_board, opening, length):
    m,n,direction = opening
    while length:
        m,n = directions_dict[direction](m,n)
        n = add_board.board[m,n]
        if n in board_values:
            length-=1
    if m not in range(0,15) or n not in range(0,15):
        return 1

In [17]:
def add_word(player, add_board, opening, tiles_indices):
    '''
    input player as string, sp as tuple, direction as 0, 1, 2, 3, tiles as list of indices for tile objects in rack list
    '''
    if check_border(add_board,opening,len(tiles_indices))==1:
        return 'You\'re trying to play a tile outside the board!'
    tiles = list()
    for i in range(len(tiles_indices)):
        tiles.append(add_board.players[player].rack[tiles_indices[i]])
    for i in tiles:
        add_board.players[player].rack.remove(i)
    bingo = 0 
    if len(tiles)==7:    bingo = 50
    score = 0
    score_multiply = 1
    i,j,direction = opening
    p = directions_dict[reverse_dict[direction]]
    while read_board.board[i,j] not in board_values:
        i,j = p(i,j)
        if i not in range(0,15) or j not in range(0,15):
            break
    i,j = directions_dict[direction](i,j)
    while tiles:
        n = add_board.board[i,j]
        if n not in board_values:
            score+=add_board.id_dict[n].score
        else:
            if n == 0:
                score+=tiles[0].score
            if n == 2000:
                score+=tiles[0].score
                score_multiply *= 2
            if n == 3000:
                score+=tiles[0].score
                score_multiply *= 3
            if n == 200:
                score+=tiles[0].score*2
            if n == 300:
                score+=tiles[0].score*3
            add_board.board_positions.append((i,j))
            add_board.board[i,j] = tiles[0].id
            tiles.pop(0)
        i,j = directions_dict[direction](i,j)
        
    
    score*=score_multiply
    score+=bingo
    add_board.players[player].score += score
    add_board.players[player].refill(add_board)

In [18]:

def calc_word(player, add_board, opening, tiles_indices):
    '''
    input player as string, sp as tuple, direction as 0, 1, 2, 3, tiles as list of indices for tile objects in rack list
    '''
    if check_border(add_board,opening,len(tiles_indices))==1:
        return None
    tiles = list()
    for i in range(len(tiles_indices)):
        tiles.append(add_board.players[player].rack[tiles_indices[i]])
    bingo = 0 
    if len(tiles)==7:    bingo = 50
    score = 0
    score_multiply = 1
    i,j,direction = opening
    p = directions_dict[reverse_dict[direction]]
    while read_board.board[i,j] not in board_values:
        i,j = p(i,j)
        if i not in range(0,15) or j not in range(0,15):
            break
    i,j = directions_dict[direction](i,j)
    while tiles:
        n = add_board.board[i,j]
        if n not in board_values:
            score+=add_board.id_dict[n].score
        else:
            if n == 0:
                score+=tiles[0].score
            if n == 2000:
                score+=tiles[0].score
                score_multiply *= 2
            if n == 3000:
                score+=tiles[0].score
                score_multiply *= 3
            if n == 200:
                score+=tiles[0].score*2
            if n == 300:
                score+=tiles[0].score*3
            tiles.pop(0)
        i,j = directions_dict[direction](i,j)
    
    score*=score_multiply
    score+=bingo
    return score

In [19]:
def compmove(playerboard, player):
    # some issue with referencing moves
    surro = find_moves(playerboard,find_openings(playerboard),player.rack)
    moves = surro
    move_scores = np.zeros((len(moves)))
    for i in range(len(moves)):
        opening, add = moves[i]
        tile_indices = list()
        for tile in add:
            tile_indices.append(player.rack.index(tile))
        move_scores[i] = calc_word(player,playerboard,opening,tile_indices)
    opt_opening, opt_add = moves[np.argmax(move_scores)]
    opt_indices = list()
    for tile in opt_add:
            opt_indices.append(player.rack.index(tile))
    add_word(player, playerboard, opt_opening, opt_indices)
    show_board(playerboard)
    
def playermove(playerboard,player):
    player.show_rack()
    print('You must start at a tile ')
    row = input('Which row would you like to start in? The rows are 1 to 15 from top to bottom: ')
    col = input('Which column would you like to start in? The columns are 1 to 15 from left to right: ')
    direction = input('Which direction would you like to go? Up is 1, Right is 2, Down is 3, and Left is 4: ')
    opening = (int(row)-1, int(col)-1, int(direction)-1)
    order = input('How do you want to order the tiles? If you\'re using a tile on the board, ignore it and enter the next tile on your own rack. Enter the order as a series of numbers from 1 to 7 with no spaces. Remember that you are starting from the chosen spot and placing in the chosen direction.')
    tile_indices = list()
    for i in order:
        tile_indices.append(int(i)-1)
    add_word(player, playerboard, opening, tile_indices)
    show_board(playerboard)

In [20]:
testboard = Board(2)
for player in testboard.players.values():
    player.refill(testboard)
while testboard.pool:
    compmove(testboard,testboard.players['Comp'])
    playermove(testboard,testboard.players['Human'])

What is Player 1's name?
Comp
What is Player 2's name?
Human


TypeError: 'Board' object is not subscriptable

In [208]:
def funcadd(item, dictionary):
    dictionary[item] = item

In [210]:
delta = ['a','b','c']
delta = delta[:len(delta)-1]
delta

['a', 'b']

In [214]:
n = 0
for i in []:
    n+=1
n

0

3619

In [160]:
225*4*3619

3619